# Eczema and Atopic dermatitis articles classification

In [ ]:
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import tree
import graphviz

## Data structure initialization

In [ ]:
tokenize = lambda doc: doc.lower().split(" ")
dictLabel = {}
labelId = 0
path = "./data/cleaned/"

labels = ["eczema", "AD"]
labelIds = [0,1]
XRawList = []
yList = []

### Function for creating balanced subsampled dataset

As the documents indexed under _dermatitis, atopic_ outnumber those indexed under _eczema_, train set need to be balanced. We use down sampling, randomly selecting a number of documents from the majority class (_dermatitis, atopic_) equals to the number of documents from the minority class. As the training set will be different at each run, results might slightly vary. 

In [ ]:
def balanced_subsample(x,y,subsample_size=1.0):

    class_xs = []
    min_elems = None

    for yi in np.unique(y):
        elems = x[(y == yi)]
        class_xs.append((yi, elems))
        if min_elems == None or elems.shape[0] < min_elems:
            min_elems = elems.shape[0]

    use_elems = min_elems
    if subsample_size < 1:
        use_elems = int(min_elems*subsample_size)

    xs = []
    ys = []

    for ci,this_xs in class_xs:
        if len(this_xs) > use_elems:
        #if this_xs.shape[0] > use_elems:
            np.random.shuffle(this_xs)

        x_ = this_xs[:use_elems]
        y_ = np.empty(use_elems)
        y_.fill(ci)

        xs.append(x_)
        ys.append(y_)

    xs = np.concatenate(xs)
    ys = np.concatenate(ys)

    return xs,ys

## Import documents

Documents correspond to preprocessed title and abstract retrieved from pubmed. Preprocessing include removal of stop words and convertion of words to lemma.

In [ ]:
for label in os.listdir(path) :

    if "Store" not in label and "ipynb_checkpoints" not in label: 

        for txt in os.listdir(path+"/"+label):
            if "ipynb_checkpoints" not in txt :
                with open(path+"/"+label+"/"+txt) as finput :
                    XRawList.append(finput.read())
                    if "derma" in label : 
                        yList.append(1)
                    else :
                        yList.append(0)


y = np.asarray(yList)
XRaw = np.asarray(XRawList)

## Convert documents to feature vectors

Documents are represented as binary vectors, for which each position correpond to a word, and the value correspond to its presence or absence in the title or abstract. Each document has a known class definied by its MeSH indexing term in PubMed.

In [ ]:
# Vectorize documents
# -- min_df : minimum frequency required to consider a word. Allow to reduce dimensions.
# -- ngram_range : allow to consider ngrams (contiguous sequence of words) as feature. When set to [1,1], 
#    consider only single words for a simplier model.
# -- binary : consider presence/absence of a word if true, number of occurences otherwise
sklearn_count = CountVectorizer(min_df=10, tokenizer=tokenize, ngram_range=[1,1], binary=True)
X = sklearn_count.fit_transform(XRaw)
# convert to dense matrix
X = X.todense()

# save vocabulary to translate features key back to words
inv_map = {v: k for k, v in sklearn_count.vocabulary_.items()}
feat_names = []
with open("dict.txt", "w") as fout : 
    for key in sorted(inv_map.keys()) : 
        fout.write(str(key)+"\t"+inv_map[key]+"\n")
        feat_names.append(inv_map[key])


## Learn Decision Tree

Build Decision Tree for _eczema_ and _dermatitis, atopic_ indexing classification, using CART algorithm with Gini impurity as split criterion. Maximum tree depth set to 6 and minimum proportion of total sample in a leaf set to 1%, in order to avoid overfitting and keep a simple model for ease of interpretation.
The test set represent 20% of the whole dataset.

In [ ]:
# Randomly split dataset into train et test set
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.2, stratify=y, random_state = 42)

# Balance trainset to avoid bias toward majority class 
X_train, y_train =  balanced_subsample(X,y,subsample_size=1.0)

# Perform decision tree learning
# -- criterion = Measure of the quality of a split, used to decide at each step 
#    which feature is best to split the data
# -- max_depth = Maximum depth of the tree.
# -- min_samples_leaf = The minimum proportion of total sample in a leaf. 
#    Below this value, no further split are performed. Prevent over-fitting
clf = tree.DecisionTreeClassifier(criterion="gini", max_depth=6, min_samples_leaf=0.01)
clf.fit(X_train,y_train)

### Create tree visualization

In [ ]:
#export tree vizualisation
with open('./viz/tree.dot', "w") as f : 
    f = tree.export_graphviz(clf,out_file=f, 
                             proportion=True, 
                             feature_names=feat_names, 
                             class_names=labels, 
                             filled=True, 
                             rounded=True, 
                             rotate=True)

with open('./viz/tree.dot', "r") as f:
    dot_graph = f.read()
graphviz.Source(dot_graph)

## Evaluation

Performance evaluated using F1 score, accuracy, specificity and ROC AUC.

In [ ]:
from sklearn.metrics import f1_score
# apply classification model to test set
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred,target_names=labels))

# compute f1 score
score = f1_score(y_test, y_pred, average='weighted') 
print("F1-score = ",(score*100))

# compute ROC curve
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

y_prob = clf.predict_proba(X_test)
fpr, tpr, _ = roc_curve(y_test, y_prob[:, 1],pos_label=1)
plt.plot(fpr, tpr,'r-')
plt.plot([0, 1], [0, 1],'k--')
plt.title('ROC curve')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()
# print AUC score
print("ROC AUC =",roc_auc_score(y_test, y_prob[:, 1]))